# E-commerce Product Pricing Analysis 

# Pricing Intelligence Analysis of Women's Kurtas on Flipkart

Import Libraries

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import re
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime


In [3]:
# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline

# Step 1 – Web Scraping

In [25]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import re
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Set up headers to mimic a browser request
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

def scrape_flipkart_kurtas():
    base_url = "https://www.flipkart.com"
    search_url = base_url + "/search?q=women+kurtas&page={}"
    
    products_data = []
    page = 1
    max_pages = 15  # To get around 300 products
    
    while len(products_data) < 300 and page <= max_pages:
        print(f"Scraping page {page}...")
        
        try:
            response = requests.get(search_url.format(page), headers=headers)
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Find all product containers
            products = soup.find_all('div', {'class': '_1AtVbE'})
            
            for product in products:
                # Check if we have enough products
                if len(products_data) >= 300:
                    break
                    
                # Extract product details
                try:
                    # Product name
                    name_tag = product.find('a', {'class': 'IRpwTa'})
                    if not name_tag:
                        continue
                    name = name_tag.text.strip()
                    
                    # Brand
                    brand_tag = product.find('div', {'class': '_2WkVRV'})
                    brand = brand_tag.text.strip() if brand_tag else "Unknown"
                    
                    # Prices
                    price_tag = product.find('div', {'class': '_30jeq3'})
                    discounted_price = float(price_tag.text.replace('₹', '').replace(',', '')) if price_tag else None
                    
                    mrp_tag = product.find('div', {'class': '_3I9_wc'})
                    mrp = float(mrp_tag.text.replace('₹', '').replace(',', '')) if mrp_tag else discounted_price
                    
                    # Discount percentage
                    discount_tag = product.find('div', {'class': '_3Ay6Sb'})
                    discount_percent = float(discount_tag.text.replace('% off', '').replace(' off', '')) if discount_tag else 0
                    
                    # Rating
                    rating_tag = product.find('div', {'class': '_3LWZlK'})
                    rating = float(rating_tag.text) if rating_tag else None
                    
                    # Reviews count
                    reviews_tag = product.find('span', {'class': '_2_R_DZ'})
                    reviews_text = reviews_tag.text if reviews_tag else ""
                    reviews_match = re.search(r'(\d+)[^\d]*$', reviews_text)
                    reviews_count = int(reviews_match.group(1)) if reviews_match else 0
                    
                    # Product URL
                    url_tag = product.find('a', {'class': 'IRpwTa'})
                    product_url = base_url + url_tag['href'] if url_tag and 'href' in url_tag.attrs else ""
                    
                    # Category
                    category = "Women's Kurtas"
                    
                    # Add to our data list
                    products_data.append({
                        'Product Name': name,
                        'Brand': brand,
                        'Category': category,
                        'MRP': mrp,
                        'Discounted Price': discounted_price,
                        'Discount Percentage': discount_percent,
                        'Rating': rating,
                        'Number of Reviews': reviews_count,
                        'Product URL': product_url
                    })
                    
                except Exception as e:
                    print(f"Error parsing product: {e}")
                    continue
                    
            page += 1
            time.sleep(2)  # Be respectful with requests
            
        except Exception as e:
            print(f"Error scraping page {page}: {e}")
            break
            
    return products_data

# Scrape the data
print("Starting web scraping...")
products_data = scrape_flipkart_kurtas()
print(f"Scraped {len(products_data)} products")

# Create DataFrame
df = pd.DataFrame(products_data)
df.to_csv('flipkart_women_kurtas_raw.csv', index=False)
print("Raw data saved to flipkart_women_kurtas_raw.csv")

Starting web scraping...
Scraping page 1...
Scraping page 2...
Scraping page 3...
Scraping page 4...
Scraping page 5...
Scraping page 6...
Scraping page 7...
Scraping page 8...
Scraping page 9...
Scraping page 10...
Scraping page 11...
Scraping page 12...
Scraping page 13...
Scraping page 14...
Scraping page 15...
Scraped 0 products
Raw data saved to flipkart_women_kurtas_raw.csv


# Step 2: Data Cleaning & Preparation

In [27]:
# Data Cleaning Function
def clean_data(df):
    # Create a copy of the dataframe
    cleaned_df = df.copy()
    
    # Remove duplicates
    print(f"Before removing duplicates: {len(cleaned_df)} rows")
    cleaned_df = cleaned_df.drop_duplicates(subset=['Product Name', 'Brand'])
    print(f"After removing duplicates: {len(cleaned_df)} rows")
    
    # Handle missing values
    # For numeric columns, we'll fill with appropriate values
    cleaned_df['MRP'] = cleaned_df['MRP'].fillna(cleaned_df['Discounted Price'])
    cleaned_df['Discounted Price'] = cleaned_df['Discounted Price'].fillna(cleaned_df['MRP'])
    
    # Calculate discount percentage if missing
    mask = (cleaned_df['Discount Percentage'] == 0) & (cleaned_df['MRP'] > cleaned_df['Discounted Price'])
    cleaned_df.loc[mask, 'Discount Percentage'] = round(
        ((cleaned_df.loc[mask, 'MRP'] - cleaned_df.loc[mask, 'Discounted Price']) / cleaned_df.loc[mask, 'MRP']) * 100, 2
    )
    
    # Fill missing ratings with 0 (assuming no rating)
    cleaned_df['Rating'] = cleaned_df['Rating'].fillna(0)
    
    # Fill missing reviews with 0
    cleaned_df['Number of Reviews'] = cleaned_df['Number of Reviews'].fillna(0)
    
    # Standardize brand names (convert to uppercase and remove extra spaces)
    cleaned_df['Brand'] = cleaned_df['Brand'].str.upper().str.strip()
    
    # Remove any rows where essential information is missing
    cleaned_df = cleaned_df.dropna(subset=['Product Name', 'MRP', 'Discounted Price'])
    
    return cleaned_df

# Load the raw data (if not already in memory)
try:
    df
except NameError:
    df = pd.read_csv('flipkart_women_kurtas_raw.csv')

# Clean the data with error handling
try:
    # First, check what columns we actually have
    print("Available columns in raw data:")
    print(df.columns.tolist())
    print(f"DataFrame shape: {df.shape}")
    
    # Check if we have the essential columns for cleaning
    essential_columns = ['Product Name', 'Brand']
    available_essential = [col for col in essential_columns if col in df.columns]
    
    if not available_essential:
        print("Warning: No essential columns found. Data cannot be cleaned properly.")
        cleaned_df = df.copy()
    else:
        print(f"Essential columns found: {available_essential}")
        cleaned_df = clean_data(df)
    
    # Save cleaned data
    cleaned_df.to_csv('flipkart_women_kurtas_cleaned.csv', index=False)
    print("Cleaned data saved to flipkart_women_kurtas_cleaned.csv")
    
    # Display basic info about the cleaned data
    print("\nCleaned Data Info:")
    print(f"Shape: {cleaned_df.shape}")
    print(f"Columns: {cleaned_df.columns.tolist()}")
    
    print("\nData types:")
    print(cleaned_df.dtypes)
    
    print("\nFirst 5 rows:")
    print(cleaned_df.head())
    
    print("\nMissing values:")
    print(cleaned_df.isnull().sum())
    
except Exception as e:
    print(f"Error during data cleaning: {e}")
    print("Saving raw data as cleaned data due to errors...")
    df.to_csv('flipkart_women_kurtas_cleaned.csv', index=False)
    cleaned_df = df

Available columns in raw data:
[]
DataFrame shape: (0, 0)
Cleaned data saved to flipkart_women_kurtas_cleaned.csv

Cleaned Data Info:
Shape: (0, 0)
Columns: []

Data types:
Series([], dtype: object)

First 5 rows:
Empty DataFrame
Columns: []
Index: []

Missing values:
Series([], dtype: float64)


# Step 3: Data Analysis

In [30]:
# Data Analysis
def analyze_data(df):
    analysis_results = {}
    
    # First, let's check what columns we actually have
    print("Available columns in the DataFrame:")
    print(df.columns.tolist())
    
    # Check if required columns exist, if not use alternatives
    price_columns = []
    if 'MRP' in df.columns:
        price_columns.append('MRP')
    if 'Discounted Price' in df.columns:
        price_columns.append('Discounted Price')
    
    # If we don't have the expected column names, look for similar ones
    if not price_columns:
        for col in df.columns:
            if 'price' in col.lower() or 'mrp' in col.lower():
                price_columns.append(col)
    
    # If still no price columns, use the first numeric column
    if not price_columns:
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        if len(numeric_cols) > 0:
            price_columns = [numeric_cols[0]]
        else:
            print("No numeric columns found for price analysis")
            return analysis_results, df
    
    # Descriptive statistics for available price columns
    analysis_results['price_stats'] = df[price_columns].describe()
    
    # Rating statistics (handle missing rating column)
    if 'Rating' in df.columns:
        analysis_results['rating_stats'] = df['Rating'].describe()
    else:
        rating_col = None
        for col in df.columns:
            if 'rating' in col.lower():
                rating_col = col
                break
        if rating_col:
            analysis_results['rating_stats'] = df[rating_col].describe()
        else:
            print("Rating column not found")
    
    # Reviews statistics (handle missing reviews column)
    if 'Number of Reviews' in df.columns:
        analysis_results['reviews_stats'] = df['Number of Reviews'].describe()
    else:
        reviews_col = None
        for col in df.columns:
            if 'review' in col.lower() or 'rating' in col.lower():
                reviews_col = col
                break
        if reviews_col:
            analysis_results['reviews_stats'] = df[reviews_col].describe()
        else:
            print("Reviews column not found")
    
    # Brand analysis - Top 5 brands by number of products
    if 'Brand' in df.columns:
        brand_counts = df['Brand'].value_counts().head(5)
        analysis_results['top_brands'] = brand_counts
    else:
        brand_col = None
        for col in df.columns:
            if 'brand' in col.lower():
                brand_col = col
                break
        if brand_col:
            brand_counts = df[brand_col].value_counts().head(5)
            analysis_results['top_brands'] = brand_counts
        else:
            print("Brand column not found")
    
    # Discount analysis - Brands with highest average discount
    if 'Discount Percentage' in df.columns and 'Brand' in df.columns:
        brand_discounts = df.groupby('Brand')['Discount Percentage'].mean().sort_values(ascending=False)
        analysis_results['top_discount_brands'] = brand_discounts.head(5)
    
    # Price range analysis (use the first available price column)
    if price_columns:
        price_col = price_columns[0]
        price_bins = [0, 500, 1000, 1500, 2000, 3000, 5000, 10000, float('inf')]
        price_labels = ['<500', '500-1000', '1000-1500', '1500-2000', '2000-3000', '3000-5000', '5000-10000', '>10000']
        df['Price Range'] = pd.cut(df[price_col], bins=price_bins, labels=price_labels)
        analysis_results['price_range_dist'] = df['Price Range'].value_counts()
    
    # Rating distribution
    rating_col = 'Rating' if 'Rating' in df.columns else None
    if not rating_col:
        for col in df.columns:
            if 'rating' in col.lower():
                rating_col = col
                break
    
    if rating_col:
        rating_bins = [0, 1, 2, 3, 4, 5]
        rating_labels = ['0-1', '1-2', '2-3', '3-4', '4-5']
        df['Rating Range'] = pd.cut(df[rating_col], bins=rating_bins, labels=rating_labels)
        analysis_results['rating_dist'] = df['Rating Range'].value_counts()
    
    # Additional analysis: Correlation between price and rating
    if price_columns and rating_col:
        analysis_results['price_rating_corr'] = df[price_col].corr(df[rating_col])
    
    # Additional analysis: Average rating by price range
    if 'Price Range' in df.columns and rating_col:
        analysis_results['avg_rating_by_price'] = df.groupby('Price Range')[rating_col].mean()
    
    return analysis_results, df

# Perform analysis
analysis_results, analyzed_df = analyze_data(cleaned_df)

# Print analysis results
print("\n" + "="*50)
print("DATA ANALYSIS RESULTS")
print("="*50)

if 'price_stats' in analysis_results:
    print("\n=== PRICE STATISTICS ===")
    print(analysis_results['price_stats'])

if 'rating_stats' in analysis_results:
    print("\n=== RATING STATISTICS ===")
    print(analysis_results['rating_stats'])

if 'reviews_stats' in analysis_results:
    print("\n=== REVIEWS STATISTICS ===")
    print(analysis_results['reviews_stats'])

if 'top_brands' in analysis_results:
    print("\n=== TOP 5 BRANDS BY PRODUCT COUNT ===")
    print(analysis_results['top_brands'])

if 'top_discount_brands' in analysis_results:
    print("\n=== TOP 5 BRANDS BY AVERAGE DISCOUNT ===")
    print(analysis_results['top_discount_brands'])

if 'price_range_dist' in analysis_results:
    print("\n=== PRICE RANGE DISTRIBUTION ===")
    print(analysis_results['price_range_dist'])

if 'rating_dist' in analysis_results:
    print("\n=== RATING DISTRIBUTION ===")
    print(analysis_results['rating_dist'])

if 'price_rating_corr' in analysis_results:
    print("\n=== ADDITIONAL INSIGHTS ===")
    print(f"\nCorrelation between Price and Rating: {analysis_results['price_rating_corr']:.3f}")

if 'avg_rating_by_price' in analysis_results:
    print("\nAverage Rating by Price Range:")
    print(analysis_results['avg_rating_by_price'])

# Save the analyzed dataframe
analyzed_df.to_csv('flipkart_women_kurtas_analyzed.csv', index=False)
print("\nAnalyzed data saved to flipkart_women_kurtas_analyzed.csv")

Available columns in the DataFrame:
[]
No numeric columns found for price analysis

DATA ANALYSIS RESULTS

Analyzed data saved to flipkart_women_kurtas_analyzed.csv


# Step 4: Data Visualization

In [32]:
# Set up the visualization style
plt.style.use('default')
sns.set_palette("husl")

# Create visualizations
def create_visualizations(df):
    # First, identify the actual column names in the dataframe
    print("Available columns for visualization:")
    print(df.columns.tolist())
    
    # Find the appropriate column names
    price_col = None
    rating_col = None
    discount_col = None
    brand_col = None
    
    for col in df.columns:
        col_lower = col.lower()
        if 'price' in col_lower and ('discount' in col_lower or 'sale' in col_lower):
            price_col = col
        elif 'rating' in col_lower:
            rating_col = col
        elif 'discount' in col_lower and 'percentage' in col_lower:
            discount_col = col
        elif 'discount' in col_lower:
            discount_col = col
        elif 'brand' in col_lower:
            brand_col = col
    
    # If we couldn't find the exact columns, use the first available ones
    if not price_col:
        numeric_cols = df.select_dtypes(include=[np.number]).columns
        if len(numeric_cols) > 0:
            price_col = numeric_cols[0]
    
    if not brand_col:
        for col in df.columns:
            if df[col].dtype == 'object' and df[col].nunique() < 50:  # Likely a categorical column like brand
                brand_col = col
                break
    
    print(f"\nUsing columns - Price: {price_col}, Rating: {rating_col}, Discount: {discount_col}, Brand: {brand_col}")
    
    # 1. Histogram - Distribution of product prices
    if price_col:
        plt.figure(figsize=(12, 7))
        plt.hist(df[price_col].dropna(), bins=30, edgecolor='black', alpha=0.7, color='skyblue')
        plt.title('Distribution of Product Prices (Women\'s Kurtas)', fontsize=16, fontweight='bold')
        plt.xlabel('Price (₹)', fontsize=12)
        plt.ylabel('Frequency', fontsize=12)
        plt.grid(axis='y', alpha=0.75)
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig('price_distribution_histogram.png', dpi=300, bbox_inches='tight')
        plt.show()
    else:
        print("Skipping price histogram - no suitable price column found")
    
    # 2. Bar chart - Average discount percentage by brand (top brands)
    if brand_col and discount_col:
        # Get top brands (at least 5 products)
        brand_counts = df[brand_col].value_counts()
        top_brands = brand_counts[brand_counts >= 5].head(10).index
        
        if len(top_brands) > 0:
            top_brands_data = df[df[brand_col].isin(top_brands)]
            brand_discounts = top_brands_data.groupby(brand_col)[discount_col].mean().sort_values(ascending=False)
            
            plt.figure(figsize=(14, 8))
            colors = plt.cm.Set3(np.linspace(0, 1, len(brand_discounts)))
            bars = plt.bar(range(len(brand_discounts)), brand_discounts.values, color=colors)
            
            plt.title('Average Discount Percentage by Brand', fontsize=16, fontweight='bold')
            plt.xlabel('Brand', fontsize=12)
            plt.ylabel('Average Discount Percentage', fontsize=12)
            plt.xticks(range(len(brand_discounts)), brand_discounts.index, rotation=45, ha='right')
            plt.grid(axis='y', alpha=0.75)
            
            # Add value labels on bars
            for i, bar in enumerate(bars):
                height = bar.get_height()
                plt.text(bar.get_x() + bar.get_width()/2., height + 0.5,
                        f'{height:.1f}%', ha='center', va='bottom', fontweight='bold')
            
            plt.tight_layout()
            plt.savefig('discount_by_brand_barchart.png', dpi=300, bbox_inches='tight')
            plt.show()
        else:
            print("Skipping brand discount chart - not enough brands with sufficient products")
    else:
        print("Skipping brand discount chart - missing brand or discount column")
    
    # 3. Box plot - Price distribution across the category
    if price_col:
        plt.figure(figsize=(12, 7))
        sns.boxplot(y=df[price_col].dropna())
        plt.title('Price Distribution of Women\'s Kurtas', fontsize=16, fontweight='bold')
        plt.ylabel('Price (₹)', fontsize=12)
        plt.grid(axis='y', alpha=0.75)
        plt.tight_layout()
        plt.savefig('price_distribution_boxplot.png', dpi=300, bbox_inches='tight')
        plt.show()
    else:
        print("Skipping price box plot - no suitable price column found")
    
    # 4. Scatter plot - Ratings vs Discount Percentage
    if rating_col and discount_col:
        plt.figure(figsize=(12, 7))
        plt.scatter(df[rating_col], df[discount_col], alpha=0.6, c='purple', s=50)
        plt.title('Ratings vs Discount Percentage', fontsize=16, fontweight='bold')
        plt.xlabel('Rating', fontsize=12)
        plt.ylabel('Discount Percentage', fontsize=12)
        plt.grid(alpha=0.75)
        
        # Add trend line if there's enough data
        if len(df.dropna(subset=[rating_col, discount_col])) > 2:
            z = np.polyfit(df[rating_col], df[discount_col], 1)
            p = np.poly1d(z)
            plt.plot(df[rating_col], p(df[rating_col]), "r--", alpha=0.8, linewidth=2)
        
        plt.tight_layout()
        plt.savefig('ratings_vs_discount_scatter.png', dpi=300, bbox_inches='tight')
        plt.show()
    else:
        print("Skipping ratings vs discount scatter plot - missing rating or discount column")
    
    # Additional visualization: Price vs Rating
    if price_col and rating_col:
        plt.figure(figsize=(12, 7))
        plt.scatter(df[price_col], df[rating_col], alpha=0.6, c='green', s=50)
        plt.title('Price vs Rating', fontsize=16, fontweight='bold')
        plt.xlabel('Price (₹)', fontsize=12)
        plt.ylabel('Rating', fontsize=12)
        plt.grid(alpha=0.75)
        
        # Add trend line if there's enough data
        if len(df.dropna(subset=[price_col, rating_col])) > 2:
            z = np.polyfit(df[price_col], df[rating_col], 1)
            p = np.poly1d(z)
            plt.plot(df[price_col], p(df[price_col]), "r--", alpha=0.8, linewidth=2)
        
        plt.tight_layout()
        plt.savefig('price_vs_rating_scatter.png', dpi=300, bbox_inches='tight')
        plt.show()
    else:
        print("Skipping price vs rating scatter plot - missing price or rating column")
    
    # Additional visualization: Price distribution by brand (for top brands)
    if price_col and brand_col:
        brand_counts = df[brand_col].value_counts()
        top_brands = brand_counts[brand_counts >= 3].head(8).index  # Brands with at least 3 products
        
        if len(top_brands) > 0:
            top_brands_data = df[df[brand_col].isin(top_brands)]
            
            plt.figure(figsize=(14, 8))
            sns.boxplot(x=brand_col, y=price_col, data=top_brands_data)
            plt.title('Price Distribution by Brand', fontsize=16, fontweight='bold')
            plt.xlabel('Brand', fontsize=12)
            plt.ylabel('Price (₹)', fontsize=12)
            plt.xticks(rotation=45, ha='right')
            plt.grid(axis='y', alpha=0.75)
            plt.tight_layout()
            plt.savefig('price_by_brand_boxplot.png', dpi=300, bbox_inches='tight')
            plt.show()

# Create visualizations
create_visualizations(analyzed_df)

Available columns for visualization:
[]

Using columns - Price: None, Rating: None, Discount: None, Brand: None
Skipping price histogram - no suitable price column found
Skipping brand discount chart - missing brand or discount column
Skipping price box plot - no suitable price column found
Skipping ratings vs discount scatter plot - missing rating or discount column
Skipping price vs rating scatter plot - missing price or rating column
